In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
import logging
logging.basicConfig(level=logging.ERROR)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
max_length = 300
batch_size = 4
epochs = 3

df = pd.read_csv('Train.csv')

X = df.iloc[:, 1: 6]
y = df.iloc[:, 6: 6 + 25]

LABELS = y.columns

In [3]:
train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, test_size=.2)

train_X.shape, train_y.shape, test_X.shape, test_y.shape

((11203, 5), (11203, 25), (2801, 5), (2801, 25))

In [4]:
del train_X, train_y
X

,ABSTRACT,Computer Science,Mathematics,Physics,Statistics
0,a ever-growing datasets inside observational a...,0,0,1,0
1,we propose the framework considering optimal $...,1,0,0,0
2,nanostructures with open shell transition meta...,0,0,1,0
3,stars are self-gravitating fluids inside which...,0,0,1,0
4,deep neural perception and control networks ar...,1,0,0,0
...,...,...,...,...,...
13999,a methodology of automatic detection of a even...,1,0,0,0
14000,we consider a case inside which the robot has ...,1,0,0,0
14001,despite being usually considered two competing...,0,0,1,0
14002,we present the framework and its implementatio...,1,0,0,0


In [5]:
y

,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Vision and Pattern Recognition,Cosmology and Nongalactic Astrophysics,Data Structures and Algorithms,Differential Geometry,Earth and Planetary Astrophysics,...,Methodology,Number Theory,Optimization and Control,Representation Theory,Robotics,Social and Information Networks,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14000,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
14001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
14002,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
# train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)
# train_X = train_X.reset_index(drop=True)
test_X = test_X.reset_index(drop=True)

In [7]:
test_X.head(5)

,ABSTRACT,Computer Science,Mathematics,Physics,Statistics
0,"inside this paper, we prove the sharp limit on...",1,0,0,0
1,a assumption of separability was the simplifyi...,0,1,0,0
2,"we present clustering properties from 579,492 ...",0,0,1,0
3,we survey and prove properties the family of r...,0,1,0,0
4,opportunity detection at secondary transmitter...,1,0,0,0


In [8]:
test_y

,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Vision and Pattern Recognition,Cosmology and Nongalactic Astrophysics,Data Structures and Algorithms,Differential Geometry,Earth and Planetary Astrophysics,...,Methodology,Number Theory,Optimization and Control,Representation Theory,Robotics,Social and Information Networks,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2797,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2798,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2799,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# train_y = train_y.values
test_y = test_y.values
y = y.values
test_y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [10]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentences,
        depts,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentences = sentences
        self.depts = depts
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentences))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentences) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentences = self.sentences[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentences.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        
        dept_ids = np.array(self.depts[indexes], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
        
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, dept_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, dept_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [11]:
X.iloc[:,1:6].columns, y.shape

(Index(['Computer Science', 'Mathematics', 'Physics', 'Statistics'], dtype='object'),
 (14004, 25))

In [12]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [13]:
# strategy = tf.distribute.MirroredStrategy()

# with strategy.scope():
    
# Encoded token ids from BERT tokenizer.

def get_model():
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )

    input_departments = tf.keras.Input(
        shape = (4,), dtype=tf.float32, name='input_depts'
    )

    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )

    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )

    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")

    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )

    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)

    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool, input_departments])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(25, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, input_departments, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc", f1_m],
    )
    return model


model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 300)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 300)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 300)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 300, 768), ( 109482240   input_ids[0][0]                  
______________________________________________________________________________________________

In [14]:

val_data = BertSemanticDataGenerator(
    test_X['ABSTRACT'].values.astype("str"),
    test_X.iloc[:,1:6].values.astype("int32"),
    test_y,
    batch_size=batch_size,
    shuffle=True,
)


data = BertSemanticDataGenerator(
    X['ABSTRACT'].values.astype("str"),
    X.iloc[:,1:6].values.astype("int32"),
    y,
    batch_size=batch_size,
    shuffle=True,
)


In [15]:
X.iloc[:,1:6]

,Computer Science,Mathematics,Physics,Statistics
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,1,0,0,0
...,...,...,...,...
13999,1,0,0,0
14000,1,0,0,0
14001,0,0,1,0
14002,1,0,0,0


In [16]:
history = model.fit(
    data,
    validation_data=val_data,
    shuffle=True,
    epochs=epochs,
#     use_multiprocessing=True,
#     workers=-1,
)

437/437 [==============================] - 1478s 3s/step - loss: 2.7788 - acc: 0.3466 - f1_m: 0.2641 - val_loss: 2.3659 - val_acc: 0.4307 - val_f1_m: 0.4174


In [17]:
# # Unfreeze the bert_model.
# bert_model.trainable = True
# # Recompile the model to make the change effective.
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-5),
#     loss="categorical_crossentropy",
#     metrics=["acc", f1_m],
# )
# model.summary()

In [18]:
# history = model.fit(
#     data,
#     validation_data=val_data,
#     epochs=epochs,
#     #use_multiprocessing=True,
#     #workers=-1,
# )

In [19]:
model.evaluate(val_data, verbose=1)

87/87 [==============================] - 216s 2s/step - loss: 2.3717 - acc: 0.4303 - f1_m: 0.4173


[2.371713161468506, 0.43031609058380127, 0.41727524995803833]

In [20]:
# def check_similarity(sentence, depts):
# #     sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
#     test_data = BertSemanticDataGenerator(
#         np.array(sentence), depts, labels=None, batch_size=1, shuffle=False, include_targets=False,
#     )

#     proba = model.predict(test_data)[0]
#     idx = np.argmax(proba)
#     proba = f"{proba[idx]: .2f}%"
#     pred = labels[idx]
#     return pred, proba

In [21]:
# s = train_X.iloc[0]['ABSTRACT']
# d = train_X.iloc[:,1:6].iloc[0].values
# check_similarity(s, d)
# # sentence2 = "Two women are standing with their eyes closed."
# # check_similarity(sentence1, sentence2)

In [22]:
# sentence1 = "A smiling costumed woman is holding an umbrella"
# sentence2 = "A happy woman in a fairy costume holds an umbrella"
# check_similarity(sentence1, sentence2)

In [23]:
# sentence1 = "A soccer game with multiple males playing"
# sentence2 = "Some men are playing a sport"
# check_similarity(sentence1, sentence2)


In [24]:
# model.save("model.h5")
# print("Saved model to disk")

In [25]:
tdf = pd.read_csv("Test.csv")
# df = df.iloc[:, 1: 6]
tdf.iloc[:, 2: 6].head(5)


,Computer Science,Mathematics,Physics,Statistics
0,0,0,0,1
1,1,0,0,1
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0


In [26]:
tdf.shape

(6002, 6)

In [27]:
# train_data = BertSemanticDataGenerator(
#     train_X['ABSTRACT'].values.astype("str"),
#     train_X.iloc[:,1:6].values,
#     train_y,
#     batch_size=batch_size,
#     shuffle=True,
# )




val_data = BertSemanticDataGenerator(
    tdf['ABSTRACT'].values.astype("str"),
    tdf.iloc[:,2:6].values.astype("int32"),
    # None,
    batch_size=2,
    labels=None,
    shuffle=False, 
    include_targets=False
)




In [28]:
 = model.predict_generator(val_data, verbose=1, use_multiprocessing=True)

Instructions for updating:
Please use Model.predict, which supports generators.


Instructions for updating:
Please use Model.predict, which supports generators.


3000/3001 [============================>.] - ETA: 0sWARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


3001/3001 [==============================] - 587s 196ms/step


In [29]:
i.shape, tdf.shape

((6002, 25), (6002, 6))

In [45]:
cnt = {}
for x in y:
    ones = 0
    for xx in x:
        if xx == 1: ones += 1
    if ones in cnt: cnt[ones] += 1
    else: cnt[ones] = 1
for x in cnt: 
    print(x, cnt[x] / sum(cnt.values()))


2 0.26735218508997427
1 0.6967295058554699
3 0.03320479862896315
4 0.002713510425592688


In [31]:
i

array([[1.4769567e-12, 1.0021279e-02, 6.7738622e-02, ..., 5.4511441e-15,
        2.1791199e-15, 2.9012340e-03],
       [2.1319411e-12, 6.0804594e-02, 9.0272568e-02, ..., 5.1740842e-15,
        3.3609636e-15, 1.1984466e-02],
       [1.7668783e-04, 3.2110696e-03, 1.1257749e-04, ..., 2.4084274e-03,
        2.6943800e-03, 5.5383029e-03],
       ...,
       [6.7309087e-13, 9.0343878e-03, 1.7561410e-01, ..., 1.4999147e-15,
        4.7975094e-16, 7.8166723e-03],
       [1.9099902e-11, 1.9787643e-02, 3.6434915e-02, ..., 1.1228649e-13,
        3.3665487e-14, 8.7765604e-03],
       [3.8965602e-05, 1.8583377e-07, 4.7121652e-08, ..., 4.0466923e-01,
        2.4812873e-01, 1.6573027e-06]], dtype=float32)

In [73]:

THRESHOLD = .17

In [74]:
cnt2 = {}
for x in i:
    ones = 0
    for xx in x:
        if xx > THRESHOLD: ones += 1
    if ones in cnt2: cnt2[ones] += 1
    else: cnt2[ones] = 1
for x in cnt2: 
    print(x, cnt2[x] / sum(cnt2.values()))

1 0.44385204931689437
2 0.3915361546151283
3 0.157947350883039
4 0.00583138953682106
0 0.0008330556481172943


In [75]:
tdf.head(5)

,id,ABSTRACT,Computer Science,Mathematics,Physics,Statistics,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,...,Methodology,Number Theory,Optimization and Control,Representation Theory,Robotics,Social and Information Networks,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,9409,fundamental frequency (f0) approximation from ...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,17934,"this large-scale study, consisting of 24.5 mil...",1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16071,we present a stability analysis of the plane c...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16870,we construct finite time blow-up solutions to ...,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10496,planetary nebulae (pne) constitute an importan...,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [76]:
for col in range(len(LABELS)):
    tdf[LABELS[col]] = [1 if x[col] > THRESHOLD else 0 for x in i]
    

In [77]:
tdf.drop(columns=['ABSTRACT', 'Computer Science', 'Mathematics', 'Physics', 'Statistics']).to_csv("final.csv", index=False)


In [78]:
import pickle as pkl
pkl.dump(i, open('i.pkl', 'wb'))